In [1]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime


# Path to the dataset directory
path = r'C:\Users\Hp\Advanced Criminal Recognition\dataset'

# Load images and their corresponding names from the dataset directory
def load_dataset(path):
    images = []
    classNames = []
    for cl in os.listdir(path):
        img_path = os.path.join(path, cl)
        if os.path.isfile(img_path):
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
                classNames.append(os.path.splitext(cl)[0])
            else:
                print(f"Error: Unable to read image {img_path}")
    return images, classNames

# Function to encode faces in the dataset
def encode_faces(images):
    encodeList = []
    for img in images:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces = face_recognition.face_locations(img_rgb)
        if faces:
            encode = face_recognition.face_encodings(img_rgb, faces)[0]
            encodeList.append(encode)
    return encodeList

# Function to mark attendance by displaying the name on the frame
def detectCriminal(name, frame, x1, y1, x2, y2):
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.rectangle(frame, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
    cv2.putText(frame, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

# Load dataset
images, classNames = load_dataset(path)
if len(images) == 0:
    print("No valid images found in the dataset directory.")
    exit()

# Encode faces in the dataset
encodeListKnown = encode_faces(images)
print('Encoding Complete')

# Confidence threshold for face recognition
confidence_threshold = 0.5

# Initialize video capture
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()

    if not success:
        continue  # Skip the iteration if the frame is not successfully retrieved

    imgS = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    # Find face locations and encodings in the current frame
    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        # Compare face encodings with known encodings
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace, tolerance=0.6)
        faceDistances = face_recognition.face_distance(encodeListKnown, encodeFace)

        # Identify the best match
        matchIndex = np.argmin(faceDistances)

        if matches[matchIndex] and faceDistances[matchIndex] <= confidence_threshold:
            name = classNames[matchIndex].upper()
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            detectCriminal(name, img, x1, y1, x2, y2)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()


Encoding Complete


KeyboardInterrupt: 